In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext version_information
%version_information numpy, dlib, matplotlib, imageio, gaze, torch, torchvision

## loading the model

In [ ]:
from gaze import init, ML
args = init(no_cuda=True, batch_size=64, verbose=0)
ml = ML(args)
path = '_Regard_400.pt'
ml.train(path=path)

In [ ]:
ml.device

## running the forward model on the extracted faces

In [ ]:
%matplotlib inline
fig, ax =  ml.dataset.show()

In [ ]:
for batch_idx, (data, target) in enumerate(ml.dataset.train_loader):
    data, target = data.to(ml.device), target.to(ml.device)
    output = ml.model(data)

## testing the model

In [ ]:
Accuracy = ml.test()
print('Accuracy={:.1f}%'.format(Accuracy*100))

In [ ]:
data, labels = next(iter(ml.dataset.train_loader))
print(' '.join('%5s' % ml.dataset.dataset.classes[labels[j]] for j in range(ml.args.batch_size)))

In [ ]:
fig, ax = ml.show(only_wrong=True)

modification gaze.py ligne 343

In [ ]:
fig, ax = ml.show(only_wrong=False)

## cross-validation

In [ ]:
import os
fname = 'figures/accuracy'
if not os.path.exists(fname + '.pdf'):
    args = init(verbose=0, log_interval=0, epochs=20)
    args['N_cv'] = 5 # 20
    from gaze import MetaML
    mml = MetaML(args)
    Accuracy = mml.protocol(args, 42)
    print('Accuracy', Accuracy[:-1].mean(), '+/-', Accuracy[:-1].std())

    %matplotlib inline
    import matplotlib.pyplot as plt
    import numpy as np
    fig, ax = plt.subplots(figsize=((8, 5)))
    n, bins, patches = ax.hist(Accuracy[:-1]*100, bins=np.linspace(0, 100, 100), alpha=.4)
    ax.vlines(np.median(Accuracy[:-1])*100, 0, n.max(), 'g', linestyles='dashed', label='median')
    ax.vlines(25, 0, n.max(), 'r', linestyles='dashed', label='chance level')
    ax.vlines(100, 0, n.max(), 'k', label='max')
    ax.set_xlabel('Accuracy (%)')
    ax.set_ylabel('Smarts')
    ax.legend(loc='best')
    #plt.show() 
    for ext in ['.pdf', '.png']:
        plt.savefig(fname + ext)

In [ ]:
from IPython.display import Image
Image(filename=fname + '.png') 